In [1]:
import pandas as pd

# leer dataset 
samples_nans = pd.read_csv('../data/samples_nans.csv', sep=',')
samples_nans['date_time_init'] = pd.to_datetime(samples_nans['date_time_init'])
samples_nans['date_time_end'] = pd.to_datetime(samples_nans['date_time_end'])
samples_nans

,id_sample,id_link,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
0,0,0,100,inicio,101,primer stop,100:101,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,101,primer stop,102,segundo stop,101:102,2023-03-01 07:27:33,2023-03-01 07:29:19,106.0,13190622.0,13190922.0,SJ01,1.0,2023-03-01
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-03-01 07:29:19,2023-03-01 07:30:49,90.0,13190922.0,13191197.0,SJ01,1.0,2023-03-01
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-03-01 07:30:49,2023-03-01 07:31:13,24.0,13191197.0,13191276.0,SJ01,1.0,2023-03-01
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-03-01 07:31:13,2023-03-01 07:32:29,76.0,13191276.0,13191535.0,SJ01,1.0,2023-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278958,3360,78,178,Primero,179,Segundo,178:179,2023-03-25 11:08:40,2023-03-25 11:08:52,12.0,17265203.0,17265239.0,SJ42,2.0,2023-03-25
278959,3360,79,179,Segundo,180,Reservorio,179:180,2023-03-25 11:08:52,2023-03-25 11:09:16,24.0,17265239.0,17265310.0,SJ42,2.0,2023-03-25
278960,3360,80,180,Reservorio,181,Cuarto,180:181,2023-03-25 11:09:16,2023-03-25 11:09:28,12.0,17265310.0,17265340.0,SJ42,2.0,2023-03-25
278961,3360,81,181,Cuarto,182,Quinto,181:182,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
import numpy as np

can_nans = samples_nans['time_travel'].isna().sum()
print(f"{can_nans} nans de {samples_nans.shape[0]} rows equivale {can_nans*100/samples_nans.shape[0]:0.0f}%")

41122 nans de 278963 rows equivale 15%


### Reemplazar NaNs con promedios

In [9]:
from more_itertools import consecutive_groups

def intervals(numeros):
    intervals = []
    for group in consecutive_groups(numeros):
        group = list(group)
        if len(group) == 1:
            intervals.append(tuple(group))
        else:
            intervals.append(tuple(group))

    return intervals

In [10]:
samples_nans = samples_nans[samples_nans['id_sample'] < 5]

In [12]:
from datetime import datetime, timedelta

means_dict = samples_nans.groupby('link', observed=False).agg({'time_travel':[np.mean]})[('time_travel', 'mean')].to_list()

list_df_listos = [] 
for id, df_sample in samples_nans.groupby('id_sample'):
    # identificar los intervalos con nans
    df_sample.reset_index(inplace=True)
    index_nans = df_sample.index[pd.isna(df_sample['time_travel'])]
    inter_nans = intervals(list(index_nans))
    df_sample.insert(loc=2, column='id_linkref', value=df_sample.index)
    print(inter_nans)

    for gap in inter_nans:
        # generar los time travel
        means = [means_dict[i] for i in gap]

        # si hay nans en el medio, ajustamos means
        if gap[0] != 0 and gap[-1] != 82:
            time_init = df_sample.loc[gap[0] - 1, 'date_time_end']
            time_end = df_sample.loc[gap[-1] + 1, 'date_time_init']
            dif_time =  time_end - time_init
            means = list(map(lambda e: dif_time.total_seconds() * (e / sum(means)), means))

        # colocamos means
        for i in range(len(gap)):
            df_sample.loc[gap[i], 'time_travel'] = round(means[i], 0)
            if 0 not in gap:
                df_sample.loc[gap[i], 'date_time_init'] = df_sample.loc[gap[i]-1, 'date_time_end']
                df_sample.loc[gap[i], 'date_time_end'] = df_sample.loc[gap[i], 'date_time_init'] + timedelta(seconds=round(means[i], 0))
            else:
                j = (i+1)*(-1)
                df_sample.loc[gap[j], 'date_time_end'] = df_sample.loc[gap[j]+1, 'date_time_init']
                df_sample.loc[gap[j], 'date_time_init'] = df_sample.loc[gap[j], 'date_time_end'] - timedelta(seconds=round(means[i], 0))

    df_sample.drop(columns=['index', 'id_link'], inplace=True)                
    list_df_listos.append(df_sample)


[(0,), (15, 16), (24, 25)]
[(61, 62), (65, 66)]
[(21, 22, 23, 24, 25), (28, 29), (56, 57), (78, 79)]
[(0, 1, 2, 3), (30, 31), (59, 60), (78, 79), (82,)]
[(0,), (4, 5), (21, 22), (63, 64, 65), (73, 74), (76, 77, 78)]


In [3]:
# Extraer los samples con tiempo que sobrepasan el min y max permitido
min = 10
max = 1500
list_err = []
listo = []
# samplesdf = pd.concat(list_df_listos)
samplesdf = samples_nans.copy()
for id, sampledf in samplesdf.groupby('id_sample'):
    min_sample = sampledf['time_travel'].min()
    max_sample = sampledf['time_travel'].max()
    if (min_sample < min) or (max_sample > max):
        list_err.append(sampledf)
    else:
        listo.append(sampledf)

# Mostrar
print(f"{len(listo)} samples correctos")
print(f"{len(list_err)} samples con max, min trave times erroneos")

118 samples correctos
0 samples con max, min trave times erroneos


In [4]:
samplesdf

,id_sample,id_link,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,veh,lap,date
0,0,0,100,inicio,101,primer stop,100:101,2023-07-06 08:56:58,2023-07-06 08:58:25,87.0,SJ01,2.0,2023-07-06
1,0,1,101,primer stop,102,segundo stop,101:102,2023-07-06 08:58:25,2023-07-06 08:59:25,60.0,SJ01,2.0,2023-07-06
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-07-06 08:59:25,2023-07-06 09:00:29,64.0,SJ01,2.0,2023-07-06
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-07-06 09:00:29,2023-07-06 09:01:15,46.0,SJ01,2.0,2023-07-06
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-07-06 09:01:15,2023-07-06 09:02:05,50.0,SJ01,2.0,2023-07-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9789,117,78,178,Primero,179,Segundo,178:179,2023-07-06 16:13:01,2023-07-06 16:13:13,12.0,SJ43,5.0,2023-07-06
9790,117,79,179,Segundo,180,Reservorio,179:180,2023-07-06 16:13:13,2023-07-06 16:13:49,36.0,SJ43,5.0,2023-07-06
9791,117,80,180,Reservorio,181,Cuarto,180:181,2023-07-06 16:13:49,2023-07-06 16:14:01,12.0,SJ43,5.0,2023-07-06
9792,117,81,181,Cuarto,182,Quinto,181:182,2023-07-06 16:14:01,2023-07-06 16:14:13,12.0,SJ43,5.0,2023-07-06


In [5]:
# verificar maximos y minimos
data = pd.concat(listo).copy().reset_index(drop=True)
print('Minimos:')
print(data.nsmallest(10, 'time_travel')['time_travel'].to_string())
print('='*20)
print('Maximos:')
print(data.nlargest(10, 'time_travel')['time_travel'].to_string())

Minimos:
10     12.0
20     12.0
29     12.0
32     12.0
77     12.0
81     12.0
101    12.0
113    12.0
130    12.0
160    12.0
Maximos:
913     1257.0
4899    1160.0
2490     772.0
2074     696.0
6058     647.0
8051     494.0
5814     470.0
471      464.0
7858     436.0
2794     412.0


In [12]:
# NANs
# verificar si hay datos faltantes
print("Cantidad de NaNs:")
for col in data:
    nans = data[col].isna().sum()
    print(f"\tColumna {col}: {nans}")

Cantidad de NaNs:
	Columna id_sample: 0
	Columna id_link: 0
	Columna init_stop: 0
	Columna init_name: 0
	Columna end_stop: 0
	Columna end_name: 0
	Columna link: 0
	Columna date_time_init: 0
	Columna date_time_end: 0
	Columna time_travel: 0
	Columna veh: 0
	Columna lap: 0
	Columna date: 0
	Columna time: 0


In [11]:
data = data[~data['time_travel'].isna()]
data

,id_sample,id_link,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,veh,lap,date,time
0,0,0,100,inicio,101,primer stop,100:101,2023-07-06 08:56:58,2023-07-06 08:58:25,87.0,SJ01,2.0,2023-07-06,08:00:00
1,0,1,101,primer stop,102,segundo stop,101:102,2023-07-06 08:58:25,2023-07-06 08:59:25,60.0,SJ01,2.0,2023-07-06,08:00:00
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-07-06 08:59:25,2023-07-06 09:00:29,64.0,SJ01,2.0,2023-07-06,09:00:00
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-07-06 09:00:29,2023-07-06 09:01:15,46.0,SJ01,2.0,2023-07-06,09:00:00
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-07-06 09:01:15,2023-07-06 09:02:05,50.0,SJ01,2.0,2023-07-06,09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9789,117,78,178,Primero,179,Segundo,178:179,2023-07-06 16:13:01,2023-07-06 16:13:13,12.0,SJ43,5.0,2023-07-06,16:00:00
9790,117,79,179,Segundo,180,Reservorio,179:180,2023-07-06 16:13:13,2023-07-06 16:13:49,36.0,SJ43,5.0,2023-07-06,16:00:00
9791,117,80,180,Reservorio,181,Cuarto,180:181,2023-07-06 16:13:49,2023-07-06 16:14:01,12.0,SJ43,5.0,2023-07-06,16:00:00
9792,117,81,181,Cuarto,182,Quinto,181:182,2023-07-06 16:14:01,2023-07-06 16:14:13,12.0,SJ43,5.0,2023-07-06,16:00:00


### Poner intervalos de tiempo

In [13]:
import pandas as pd
from collections import Counter
from datetime import datetime, time, timedelta

# Convertir las columnas de tiempo en datetime
data['date_time_init'] = pd.to_datetime(data['date_time_init'])
data['date_time_end'] = pd.to_datetime(data['date_time_end'])

# aproximar a horas las datetime (para hacer el histograma)
ix = pd.DatetimeIndex(data['date_time_end']).floor('1h')
data['time'] = ix
data['time'] = data['time'].apply(lambda dt: str(dt).split()[-1]) #

# Extraer la hora en otro columna
data['hour'] = data['date_time_end'].apply(lambda dt: dt.time().strftime("%H:%M:%S"))
data.head()

,id_sample,id_link,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,veh,lap,date,time,hour
0,0,0,100,inicio,101,primer stop,100:101,2023-07-06 08:56:58,2023-07-06 08:58:25,87.0,SJ01,2.0,2023-07-06,08:00:00,08:58:25
1,0,1,101,primer stop,102,segundo stop,101:102,2023-07-06 08:58:25,2023-07-06 08:59:25,60.0,SJ01,2.0,2023-07-06,08:00:00,08:59:25
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-07-06 08:59:25,2023-07-06 09:00:29,64.0,SJ01,2.0,2023-07-06,09:00:00,09:00:29
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-07-06 09:00:29,2023-07-06 09:01:15,46.0,SJ01,2.0,2023-07-06,09:00:00,09:01:15
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-07-06 09:01:15,2023-07-06 09:02:05,50.0,SJ01,2.0,2023-07-06,09:00:00,09:02:05


In [14]:
def redondear_hora(dt, split):
    m = int(round(dt.time().minute/split, 0))*split
    if m == 60 or dt.time().hour == 4: 
        return time(dt.time().hour + 1, 0)   
    else:
        return time(dt.time().hour, m)   

#  Aproximar la 'hora' en intervalos de tiempo n minutos
n = 15
name_col = 'rg' + str(n)+'min'
data[name_col] = data['date_time_end'].apply(redondear_hora, split=n)
data.head(85)

,id_sample,id_link,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,veh,lap,date,time,hour,rg15min
0,0,0,100,inicio,101,primer stop,100:101,2023-07-06 08:56:58,2023-07-06 08:58:25,87.0,SJ01,2.0,2023-07-06,08:00:00,08:58:25,09:00:00
1,0,1,101,primer stop,102,segundo stop,101:102,2023-07-06 08:58:25,2023-07-06 08:59:25,60.0,SJ01,2.0,2023-07-06,08:00:00,08:59:25,09:00:00
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-07-06 08:59:25,2023-07-06 09:00:29,64.0,SJ01,2.0,2023-07-06,09:00:00,09:00:29,09:00:00
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-07-06 09:00:29,2023-07-06 09:01:15,46.0,SJ01,2.0,2023-07-06,09:00:00,09:01:15,09:00:00
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-07-06 09:01:15,2023-07-06 09:02:05,50.0,SJ01,2.0,2023-07-06,09:00:00,09:02:05,09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,1,4,104,Puente Huaccoto,105,Kayra,104:105,2023-07-06 12:35:50,2023-07-06 12:36:43,53.0,SJ01,3.0,2023-07-06,12:00:00,12:36:43,12:30:00
88,1,5,105,Kayra,106,Forestal,105:106,2023-07-06 12:36:43,2023-07-06 12:37:07,24.0,SJ01,3.0,2023-07-06,12:00:00,12:37:07,12:30:00
89,1,6,106,Forestal,107,Grifo Tapia,106:107,2023-07-06 12:37:07,2023-07-06 12:37:31,24.0,SJ01,3.0,2023-07-06,12:00:00,12:37:31,12:30:00
90,1,7,107,Grifo Tapia,108,Posta,107:108,2023-07-06 12:37:31,2023-07-06 12:37:55,24.0,SJ01,3.0,2023-07-06,12:00:00,12:37:55,12:30:00


In [15]:
# dividir las horas y minutos en diferentes columnas
data['horas'] = data['rg15min'].apply(lambda t: t.hour)
data['minute'] = data['rg15min'].apply(lambda t: t.minute)
data

,id_sample,id_link,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,veh,lap,date,time,hour,rg15min,horas,minute
0,0,0,100,inicio,101,primer stop,100:101,2023-07-06 08:56:58,2023-07-06 08:58:25,87.0,SJ01,2.0,2023-07-06,08:00:00,08:58:25,09:00:00,9,0
1,0,1,101,primer stop,102,segundo stop,101:102,2023-07-06 08:58:25,2023-07-06 08:59:25,60.0,SJ01,2.0,2023-07-06,08:00:00,08:59:25,09:00:00,9,0
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-07-06 08:59:25,2023-07-06 09:00:29,64.0,SJ01,2.0,2023-07-06,09:00:00,09:00:29,09:00:00,9,0
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-07-06 09:00:29,2023-07-06 09:01:15,46.0,SJ01,2.0,2023-07-06,09:00:00,09:01:15,09:00:00,9,0
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-07-06 09:01:15,2023-07-06 09:02:05,50.0,SJ01,2.0,2023-07-06,09:00:00,09:02:05,09:00:00,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9789,117,78,178,Primero,179,Segundo,178:179,2023-07-06 16:13:01,2023-07-06 16:13:13,12.0,SJ43,5.0,2023-07-06,16:00:00,16:13:13,16:15:00,16,15
9790,117,79,179,Segundo,180,Reservorio,179:180,2023-07-06 16:13:13,2023-07-06 16:13:49,36.0,SJ43,5.0,2023-07-06,16:00:00,16:13:49,16:15:00,16,15
9791,117,80,180,Reservorio,181,Cuarto,180:181,2023-07-06 16:13:49,2023-07-06 16:14:01,12.0,SJ43,5.0,2023-07-06,16:00:00,16:14:01,16:15:00,16,15
9792,117,81,181,Cuarto,182,Quinto,181:182,2023-07-06 16:14:01,2023-07-06 16:14:13,12.0,SJ43,5.0,2023-07-06,16:00:00,16:14:13,16:15:00,16,15


In [146]:
import numpy as np
means_dict = data.groupby('link', observed=False).agg({'time_travel':[np.mean]})[('time_travel', 'mean')]
std_dict = data.groupby('link', observed=False).agg({'time_travel':[np.std]})[('time_travel', 'std')]
print('promedio de travel time por link:', data['time_travel'].mean(),np.mean(means_dict.to_list()))
li = '156:157'
means_dict[li], std_dict[li]

promedio de travel time por link: 59.520059215396 59.32441171693339


(175.11764705882354, 91.8893500959955)

In [144]:
id = data[data['horas'] == 14]['id_sample'].unique()
print(id)
tmp = np.random.choice(id)
dftmp = data[data['id_sample'] == tmp]
c = ['init_name', 'end_name', 'init_stop', 'end_stop', 'horas', 'minute', 'time_travel', 'id_sample']
print(dftmp[c].to_string())

[  4  14  18  22  26  30  37  40  44  48  51  54  57  62  66  73  79  83
  87  91  95  99 112]
                      init_name                   end_name  init_stop  end_stop  horas  minute  time_travel  id_sample
7225            Puente Huaccoto                      Kayra        104       105     13       0         55.0         87
7226                      Kayra                   Forestal        105       106     13       0         67.0         87
7227                   Forestal                Grifo Tapia        106       107     13      15         36.0         87
7228                Grifo Tapia                      Posta        107       108     13      15         37.0         87
7229                      Posta                  Romeritos        108       109     13      15         22.0         87
7230                  Romeritos                    Tercero        109       110     13      15         36.0         87
7231                    Tercero                      Medio        110   

In [16]:
# guardar datos listos para entrenar
data.to_csv("../data/travel_time_series-prueba.csv", index=False)